# AUTO MD

In [1]:
pwd

'/mnt/c/Users/paul/works/1aki/1aki_2'

In [ ]:
! gmx

In [ ]:
! wget -O ions.mdp https://raw.githubusercontent.com/paulshamrat/ColabMDA/main/mdps/ions.mdp
! wget -O md.mdp https://raw.githubusercontent.com/paulshamrat/ColabMDA/main/mdps/md.mdp
! wget -O minim.mdp https://raw.githubusercontent.com/paulshamrat/ColabMDA/main/mdps/minim.mdp
! wget -O npt.mdp https://raw.githubusercontent.com/paulshamrat/ColabMDA/main/mdps/npt.mdp
! wget -O nvt.mdp https://raw.githubusercontent.com/paulshamrat/ColabMDA/main/mdps/nvt.mdp


In [ ]:

# Download pdb structure
! wget https://files.rcsb.org/download/1AKI.pdb

# Strip out Water
! grep -v HOH 1AKI.pdb > 1AKI_clean.pdb

# cleaned file
! gmx pdb2gmx -f 1AKI_clean.pdb -o 1AKI_processed.gro -water spce -ignh -ff amber99sb-ildn

# add box
! gmx editconf -f 1AKI_processed.gro -o 1AKI_newbox.gro -c -d 1.0 -bt cubic

# solvate
! gmx solvate -cp 1AKI_newbox.gro -cs spc216.gro -o 1AKI_solv.gro -p topol.top

# final sove
! gmx grompp -f ions.mdp -c 1AKI_solv.gro -p topol.top -o ions.tpr


#This is a trick to provide interactive options to gmx
! echo "SOL" > options
! echo " " >> options
# Now we have an atomic-level description of our system in the binary file ions.tpr. We will pass this file to genion:
# When prompted, choose group 13 "SOL" for embedding ions.
! gmx genion -s ions.tpr -o 1AKI_solv_ions.gro -p topol.top -pname NA -nname CL -neutral < options


# Assemble the binary input using grompp using [](this) input parameter file:
! gmx grompp -f minim.mdp -c 1AKI_solv_ions.gro -p topol.top -o em.tpr

#Using grompp to prepare our minimization MD
! gmx grompp -f minim.mdp -c 1AKI_solv_ions.gro -p topol.top -o em.tpr

#Run our minimization
! gmx mdrun -v -deffnm em


#This is a trick to provide interactive options to gmx
! echo "Potential" > options
! echo " " >> options

# Let's do a bit of analysis. The em.edr file contains all of the energy terms that GROMACS collects during EM. 
# You can analyze any .edr file using the GROMACS energy module:
! gmx energy -f em.edr -o potential.xvg < options
# At the prompt, type "10 0" to select Potential (10); zero (0) terminates input.


# We will call grompp and mdrun just as we did at the EM step:
! gmx grompp -f nvt.mdp -c em.gro -r em.gro -p topol.top -o nvt.tpr
! gmx mdrun -deffnm nvt

#################################


#This is a trick to provide interactive options to gmx
! echo "Temperature" > options
! echo " " >> options
# Let's analyze the temperature progression, again using energy:
! gmx energy -f nvt.edr -o temperature.xvg < options
# Type "16 0" at the prompt to select the temperature of the system and exit.


# We will call grompp and mdrun just as we did for NVT equilibration. 
! gmx grompp -f npt.mdp -c nvt.gro -r nvt.gro -t nvt.cpt -p topol.top -o npt.tpr
! gmx mdrun -deffnm npt



#This is a trick to provide interactive options to gmx
! echo "Pressure" > options
! echo " " >> options
# Let's analyze the pressure progression, again using energy:
! gmx energy -f npt.edr -o pressure.xvg < options
# Type "18 0" at the prompt to select the pressure of the system and exit. 



#This is a trick to provide interactive options to gmx
! echo "Density" > options
! echo " " >> options

# Let's take a look at density as well
! gmx energy -f npt.edr -o density.xvg < options
# this time using energy and entering "24 0" at the prompt.


# We will run a 1-ns MD simulation,
! gmx grompp -f md.mdp -c npt.gro -t npt.cpt -p topol.top -o md_0_1.tpr


# Assuming you have one GPU available, the mdrun command to make use of it is as simple as:
! gmx mdrun -deffnm md_0_1